In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily

# Set time period
start = datetime(2010, 1, 1)
end = datetime.now() - timedelta(days=1)  # Automatically set to yesterday
vancouver = Point(49.2497, -123.1193, 70)
# Create Point for Hanoi, Vietnam
hanoi = Point(21.0285, 105.8542, 15)
london = Point(51.5074, -0.1278, 11)
# Get daily data
df = Daily(london, start, end)
df = df.fetch()




In [140]:
df

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,NaN,324.0,11.8,NaN,1006.1,NaN
2010-01-02,2.7,0.5,5.4,0.0,NaN,254.0,15.0,NaN,1016.0,NaN
2010-01-03,1.9,0.1,3.2,0.0,NaN,43.0,6.4,NaN,1023.6,NaN
2010-01-04,-0.3,-2.0,1.9,0.0,NaN,280.0,8.9,NaN,1020.0,NaN
2010-01-05,1.0,-1.8,3.2,0.3,20.0,178.0,9.8,NaN,1005.9,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-11-26,5.8,4.6,7.8,0.3,NaN,209.0,9.3,24.1,1015.1,NaN
2024-11-27,5.2,1.2,8.3,16.4,NaN,347.0,15.1,38.9,1013.3,NaN
2024-11-28,1.7,-1.0,4.5,0.0,NaN,78.0,6.5,20.4,1027.9,NaN


In [141]:
df.shape
df.info

<bound method DataFrame.info of             tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2010-01-01   1.6   0.2   3.2   0.0   NaN  324.0  11.8   NaN  1006.1   NaN
2010-01-02   2.7   0.5   5.4   0.0   NaN  254.0  15.0   NaN  1016.0   NaN
2010-01-03   1.9   0.1   3.2   0.0   NaN   43.0   6.4   NaN  1023.6   NaN
2010-01-04  -0.3  -2.0   1.9   0.0   NaN  280.0   8.9   NaN  1020.0   NaN
2010-01-05   1.0  -1.8   3.2   0.3  20.0  178.0   9.8   NaN  1005.9   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2024-11-26   5.8   4.6   7.8   0.3   NaN  209.0   9.3  24.1  1015.1   NaN
2024-11-27   5.2   1.2   8.3  16.4   NaN  347.0  15.1  38.9  1013.3   NaN
2024-11-28   1.7  -1.0   4.5   0.0   NaN   78.0   6.5  20.4  1027.9   NaN
2024-11-29   5.3   3.1   8.0   0.0   NaN  151.0  14.3  27.8  1026.3   NaN
2024-11-30  10.0   6.8  11.8   0.0   NaN  178.0  11.7  24.1  1023.1   NaN

[5448

In [142]:
df.apply(pd.isnull).sum()

tavg       0
tmin       1
tmax       1
prcp      21
snow    5336
wdir       3
wspd       2
wpgt    3204
pres       5
tsun    5448
dtype: int64

In [143]:
#Tỷ lệ % các giá trị null trong từng cột
missing_percentage = df.apply(pd.isnull).sum()/df.shape[0] * 100
missing_percentage

tavg      0.000000
tmin      0.018355
tmax      0.018355
prcp      0.385463
snow     97.944200
wdir      0.055066
wspd      0.036711
wpgt     58.810573
pres      0.091777
tsun    100.000000
dtype: float64

In [144]:
# Nhận thấy 3 cột snow, wpgt,tsun toàn giá trị null nên ta sẽ bỏ các cột đó 

columns_to_drop = ['snow', 'wpgt','tsun']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


In [145]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9
...,...,...,...,...,...,...,...
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1
2024-11-27,5.2,1.2,8.3,16.4,347.0,15.1,1013.3
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9


In [146]:
# Loại bỏ các giá trị ngoại lai dựa trên phương pháp IQR (Interquartile Range)
numeric_columns = df.select_dtypes(include=['float64']).columns
# Hàm loại bỏ giá trị ngoại lai
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # Quartile 1
        Q3 = df[col].quantile(0.75)  # Quartile 3
        IQR = Q3 - Q1                # Interquartile Range
        lower_bound = Q1 - 1.5 * IQR # Giới hạn dưới
        upper_bound = Q3 + 1.5 * IQR # Giới hạn trên
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df
# Áp dụng hàm để loại bỏ giá trị ngoại lai cho các cột số liệu
df = remove_outliers(df, numeric_columns)

In [147]:
# Kiểm tra số dòng trước và sau khi loại bỏ ngoại lai
original_count = len(df)
new_count = len(df)

original_count, new_count

(4552, 4552)

In [148]:
df = df.interpolate(method='linear', limit_direction='forward', axis=0)

In [149]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9
...,...,...,...,...,...,...,...
2024-11-25,9.3,6.2,11.9,2.6,230.0,17.5,1001.3
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9


In [150]:
# Function to classify weather with multiple tags
def classify_weather(row):
    weather_classes = []
    
    if row['prcp'] > 0:
        weather_classes.append('Rainy')
    if row['wspd'] > 10:
        weather_classes.append('Windy')
    if row['tavg'] > 30:
        weather_classes.append('Hot')
    elif 20 <= row['tavg'] <= 30:
        weather_classes.append('Warm')
    elif 10 <= row['tavg'] < 20:
        weather_classes.append('Cool')
    else:
        weather_classes.append('Cold')
    
    return ', '.join(weather_classes)  # Return all classifications as a comma-separated string

# Apply the classification function to each row and create a new 'weather_class' column
df['weather_class'] = df.apply(classify_weather, axis=1)

In [151]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres,weather_class
time,,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1,"Windy, Cold"
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0,"Windy, Cold"
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6,Cold
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0,Cold
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9,"Rainy, Cold"
...,...,...,...,...,...,...,...,...
2024-11-25,9.3,6.2,11.9,2.6,230.0,17.5,1001.3,"Rainy, Windy, Cold"
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1,"Rainy, Cold"
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9,Cold


In [152]:
# Tao cot tavg de du doan bieu do moi
df['tavg_tomorrow'] = df['tavg'].shift(-1)
df = df.dropna(subset=['tavg_tomorrow'])

In [153]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres,weather_class,tavg_tomorrow
time,,,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1,"Windy, Cold",2.7
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0,"Windy, Cold",1.9
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6,Cold,-0.3
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0,Cold,1.0
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9,"Rainy, Cold",1.1
...,...,...,...,...,...,...,...,...,...
2024-11-22,1.5,-0.9,4.6,1.3,267.0,16.7,1007.6,"Rainy, Windy, Cold",9.3
2024-11-25,9.3,6.2,11.9,2.6,230.0,17.5,1001.3,"Rainy, Windy, Cold",5.8
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1,"Rainy, Cold",1.7


In [154]:
# Các chỉ số được sử dọng để dự báo thời tiết
predictors = ['tavg','tmin','tmax','prcp','wdir','wspd']
target = 'tavg_tomorrow'
np.random.seed(0)

split_data = np.split(df, [int(.7 * len(df)), int(.85 * len(df))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[predictors].to_numpy(), d[[target]].to_numpy()] for d in
                                                            split_data]

In [155]:
import math
# Khởi tạo weight và bias
def init_params(predictors):
    
    k = math.sqrt(1 / predictors)
    np.random.seed(0)
    #  Khởi tạo giá trị ngẫu nhiên của weight trong khoảng [0,1]
    weights = np.random.rand(predictors, 1) * 2 * k - k
    biases = np.ones((1, 1)) * 2 * k - k
    return  [weights, biases]

In [156]:
# Hàm dự đoán kết quả thời tiết
def forward(params, x):
    weights, biases = params
    prediction = x @ weights + biases
    return prediction



In [157]:
# Đo đô lệch giữa kết quả mô hình và giá trị thật
def mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)
# Đo độ thay đổi của weight whi thay đổi giá trị (đạo hàm)
def mse_grad(actual, predicted):
    return predicted - actual

In [158]:
# Hàm điểu chỉnh weight và bias để tối đa model
def backward(params, x, lr, grad):
    # #wl*tavg+w2*tmin+w3*tmax+ ... + b
    w_grad = (x.T / x.shape[0]) @ grad
    b_grad = np.mean(grad, axis=0)
    # Cập nhật weight và bias
    params[0] -= w_grad * lr
    params[1] -= b_grad * lr

    return params

In [172]:
lr = 1e-6
epochs = 400
params = init_params(train_x.shape[1])

# Tìm ra weight, bias phù hợp nhất
best_params = None
lowest_valid_loss = float('inf')

for i in range(epochs):
    predictions = forward(params, train_x)
    grad = mse_grad(train_y, predictions)
    params = backward(params, train_x, lr, grad)

    if i % 10 == 0:
        val_predictions = forward(params, valid_x)
        valid_loss = mse(valid_y, val_predictions)
        print(f'Epoch {i} loss: {valid_loss}')
        if valid_loss < lowest_valid_loss:
            lowest_valid_loss = valid_loss
            best_params = [param.copy() for param in params]  
print(f'Lowest validation loss: {lowest_valid_loss}')
weights, biases = best_params
print("Best weights:", weights)
print("Best biases:", biases)


Epoch 0 loss: 356.9895577136783
Epoch 10 loss: 147.34771118603138
Epoch 20 loss: 69.74635247043075
Epoch 30 loss: 41.4192609589673
Epoch 40 loss: 31.31727690634462
Epoch 50 loss: 27.854933443486438
Epoch 60 loss: 26.74794031316387
Epoch 70 loss: 26.436014286169762
Epoch 80 loss: 26.366017631774156
Epoch 90 loss: 26.351483471644723
Epoch 100 loss: 26.335858173123295
Epoch 110 loss: 26.306333228758337
Epoch 120 loss: 26.263333387188343
Epoch 130 loss: 26.21025426270764
Epoch 140 loss: 26.150365974970594
Epoch 150 loss: 26.08612423158992
Epoch 160 loss: 26.019205858671295
Epoch 170 loss: 25.95070403497484
Epoch 180 loss: 25.881313743974406
Epoch 190 loss: 25.81147043371366
Epoch 200 loss: 25.74144452304524
Epoch 210 loss: 25.671402966734625
Epoch 220 loss: 25.601448391240012
Epoch 230 loss: 25.531643626748746
Epoch 240 loss: 25.462026958598802
Epoch 250 loss: 25.392621562189767
Epoch 260 loss: 25.323441322169195
Epoch 270 loss: 25.254494414977188
Epoch 280 loss: 25.185785511943553
Epoch 2

In [173]:

date_to_predict = '2014-12-30'
today_features = df.loc[date_to_predict, ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd']].to_numpy().reshape(1, -1)

tavg_tomorrow_pred = forward(best_params, today_features)


print(f"Predicted tavg_tomorrow for {date_to_predict}: {tavg_tomorrow_pred[0, 0]:.2f}")


Predicted tavg_tomorrow for 2014-12-30: 9.05
